In [127]:
import pandas as pd
import numpy as np
import bs4 as bs
import requests
import pymysql as sql
from datetime import datetime
"""
The objective of this script is to create a database of the most sold
books weekly by scraping the bookstore page, all the data will be stored in 
a Pandas dataframe and later into a mySQL database using mysql.connector

"""

'\nThe objective of this script is to create a database of the most sold\nbooks weekly by scraping the bookstore page, all the data will be stored in \na Pandas dataframe and later into a mySQL database using mysql.connector\n\n'

In [16]:
#Requesting the data from the page
#we are given to work with

url_books = "https://cuspide.com/100-mas-vendidos/"    
req_books = requests.get(url_books,'lxml')          


#Requesting the data we are later going to use 
#to convert ARS$ to black market dolar prices in Argentina ('Dolar blue')                                                  

url_dolarblue = "https://www.cronista.com/MercadosOnline/dolar.html"     
req_dolarblue = requests.get(url_dolarblue, 'lxml')

In [49]:
url_dolarblue = "https://www.cronista.com/MercadosOnline/dolar.html" 


In [21]:
#Using scraping to get book tittles:

soup_books = bs.BeautifulSoup(req_books.text)

In [22]:
list_names_h3 = soup_books.find_all('h3',class_="name product-title woocommerce-loop-product__title")

#Saving all the titles on list_names for later creation of a df

list_names = [i.text for i in list_names_h3]

In [23]:

list_urls_div=soup_books.find_all("div", class_="box-image-top")

#Saving the urls of individual books

list_urls=[i.a['href'] for i in list_urls_div]


In [24]:
request_errors=[]
price_ar=[]
price_us=[]
page=0

#Here we scrap the price in both ARS and USD.
#Unfortunatly the official price in USD is not accesible from the general url, so its
#needed to acces every book url to get this, and slowing the proces big time.
#To do this we use the urls already scraped in list_urls
for book in list_urls:
    try:
        req_temp = requests.get(book,'lxml')
        instance= bs.BeautifulSoup(req_temp.text)
        info=instance.find("div",class_="product-info summary col-fit col entry-summary product-summary text-left form-minimal")
        
        #When going to append the values are formated so they can be used as int if necessary
        price_ar.append(info.find("bdi").text[2:].replace(".","").replace(",","."))
        price_us.append(info.find("span",attrs={"style":"font-size: 1.3em"}).text.replace(".","").replace(",","."))

        page+=1
        print(f"{page} pages scanned...")
    
    #In case of failed request the information of the error is stored in a list of lists
    #where every list contains [error, url_where_error], to be used in a table to audit errors
    except:
        requests.exceptions.HTTPError
        print(f"Could not obtain the data from '{list_names[page]}'")
        request_errors.append([f"Failed capture of book '{list_names[page]}'", book])

1 pages scanned...
2 pages scanned...
3 pages scanned...
4 pages scanned...
5 pages scanned...
6 pages scanned...
7 pages scanned...
8 pages scanned...
9 pages scanned...
10 pages scanned...
11 pages scanned...
12 pages scanned...
13 pages scanned...
14 pages scanned...
15 pages scanned...
16 pages scanned...
17 pages scanned...
18 pages scanned...
19 pages scanned...
20 pages scanned...
21 pages scanned...
22 pages scanned...
23 pages scanned...
24 pages scanned...
25 pages scanned...
26 pages scanned...
27 pages scanned...
28 pages scanned...
29 pages scanned...
30 pages scanned...
31 pages scanned...
32 pages scanned...
33 pages scanned...
34 pages scanned...
35 pages scanned...
36 pages scanned...
37 pages scanned...
38 pages scanned...
39 pages scanned...
40 pages scanned...
41 pages scanned...
42 pages scanned...
43 pages scanned...
44 pages scanned...
45 pages scanned...
46 pages scanned...
47 pages scanned...
48 pages scanned...
49 pages scanned...
50 pages scanned...
51 pages 

In [26]:
#Having the ARS price we can convert to 'Dolar blue' now

price_blue = []
try:
    inst_blue = bs.BeautifulSoup(req_dolarblue.text)
    aux = inst_blue.find_all("a", href="/MercadosOnline/moneda.html?id=ARSB")

    for i in aux:
        
        #I find this to be the shortest path to get the value of blue, some formating is 
        #used to clean the text,
        if ('blue' and  '$') in i.text.lower():
            i=i.text
            blue=int(i[(i.find('$')+1):i.find(',')])
            break
        
    #Divide ARS by BLUE to convert prices       
    for pr in price_ar:

        price_blue.append(round(float(pr)/blue,2))


#Storing eventual errors as before
except:
    requests.exceptions.HTTPError
    print(f"Could not obtain the data from {req_dolarblue}")
    request_errors.append([f"Failed capture of data", req_dolarblue])





In [137]:
#Now we can create the dataframe using a dicctionary based on the lists we been creating:

df_books = pd.DataFrame({"Name": list_names,  
            "Price ARS": price_ar, 
            "Price USD": price_us,
            "Price Dolar Blue": price_blue,
            "URL": list_urls})

today_ts = pd.Timestamp.today()
df_books['Update_date'] = today_ts
df_books['Update_date'] = pd.to_datetime(df_books['Update_date'].dt.date)

#The errors df
df_errors = pd.DataFrame({"Request error": [i[0] for i in request_errors],
                          "URL":[i[1] for i in request_errors]})

df_errors['Update_date'] = today_ts
df_errors['Update_date'] = pd.to_datetime(df_errors['Update_date'].dt.date)


In [143]:
#In order to load the data into the database I export them in a .csv file 
#to later load it in, there is most likely a more eficient way memory wise 
#but this one solves the problem quite elegantly  

csv_to_load_books = df_books.to_csv("C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/books_data.csv",sep=";")

csv_to_load_errors = df_errors.to_csv("C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/errors_data.csv",sep=";")


In [ ]:
"""Now having extracted all the data we neew we'll proced to create 
   the database to store it """

In [139]:
#First I use pymysql (as sql) to connect to mysql
# and continue programing using mysql sintaxis inside execute() method
cnx = sql.connect(
    user= 'root',           #Here we set the data we need to connect 
    passwd = '1234',        # in this example a local server
    host = '127.0.0.1',
    local_infile=True)


cur = cnx.cursor()


In [140]:
#Creating database and setting for use
cur.execute(        
    """CREATE DATABASE  IF NOT EXISTS top_books;
    """
    )

cur.execute(

    """USE top_books;
    """
)



0

In [154]:
#Creating the table to store the books
cur.execute("""

    DROP TABLE IF EXISTS books;

""")

cur.execute("""
    CREATE TABLE IF NOT EXISTS books(

    Idbook INT NOT NULL,
    Title VARCHAR(70),
    Price_ar FLOAT,
    Price_us FLOAT,
    Price_blue FLOAT,
    Url VARCHAR(100),
    Update_date DATE
    );
    
    """)

0

In [153]:
#Loading the data, we store in a .csv file created from the data frame

cur.execute("""

    SET GLOBAL local_infile = 1;

""")

cur.execute("""

    LOAD DATA LOCAL INFILE 'C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/books_data.csv'
    INTO TABLE books
    CHARACTER SET latin1
    FIELDS TERMINATED BY ';'
    LINES TERMINATED BY '\n'
    IGNORE 1 LINES
    (Idbook,Title, Price_ar, Price_us, Price_blue, Url, Update_date);

    """)



92

In [155]:
#Adding a primary key
cur.execute( """
    
    ALTER TABLE books
    ADD PRIMARY KEY (Idbook);

    """)

0

In [148]:
#Creating the table to store the books
cur.execute("""

    DROP TABLE IF EXISTS errors;
    
""")

cur.execute("""
    CREATE TABLE IF NOT EXISTS errors(

    Iderror INT NOT NULL,
    Description VARCHAR(100),
    Problematic_url Varchar(100),
    Update_date DATE
    );
    
    """)



0

In [149]:
#Loading the errors data
cur.execute("""

    LOAD DATA LOCAL INFILE 'C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/errors_data.csv'
    INTO TABLE errors
    CHARACTER SET latin1
    FIELDS TERMINATED BY ';'
    LINES TERMINATED BY '\n'
    IGNORE 1 LINES
    (Iderror, Description, Problematic_url, Update_date);

    """)

0

In [150]:
cur.execute( """

    ALTER TABLE errors
    ADD PRIMARY KEY (Iderror);

    """)

0

In [114]:
cnx.close()
#Finally we close the connection and the database is done